# 1.Install env

In [ ]:
!apt install -y swig cmake ffmpeg xvfb python3-opengl
!pip install stable-baselines3==2.0.0a5 gymnasium[box2d] huggingface_sb3 pyvirtualdisplay imageio[ffmpeg]

In [ ]:
!pip install --upgrade --force-reinstall stable-baselines3

In [3]:
!pip install pandas==2.2.2 numpy<2.1.0 --force-reinstall

/bin/bash: line 1: 2.1.0: No such file or directory


In [ ]:
!pip install 'shimmy>=2.0'

# 2.Test env and render video

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
from pyvirtualdisplay import Display
virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [ ]:
from pyvirtualdisplay import Display
import gymnasium as gym
import numpy as np
import imageio
from stable_baselines3 import PPO

display = Display(visible=0, size=(1400, 900))
display.start()


In [ ]:
env = gym.make("LunarLander-v3", render_mode="rgb_array")
model = PPO("MlpPolicy", env,batch_size=64,ent_coef=0.01,gae_lambda=0.98,gamma=0.999,n_epochs=4,n_steps=1024,verbose=1,learning_rate=0.001,clip_range=0.2)

In [4]:
video_filename = "lunarlander.mp4"
def record_video(model, env, video_filename="lunarlander.mp4", num_steps=200, fps=60):
    frames = []
    obs, _ = env.reset()

    for _ in range(num_steps):
        action, _ = model.predict(obs)
        obs, _, done, _, _ = env.step(action)

        frame = env.render()
        frames.append(frame)

        if done:
            obs, _ = env.reset()

    env.close()
    imageio.mimsave(video_filename, frames, fps=fps)
    print(f"✅ Video đã được lưu tại: {video_filename}")

In [5]:
from IPython.display import HTML
from base64 import b64encode

def show_video(path):
    """ Hiển thị video trực tiếp trong Google Colab """
    with open(path, "rb") as f:
        data = f.read()
    encoded = b64encode(data).decode()
    return HTML(f'<video controls autoplay loop><source src="data:video/mp4;base64,{encoded}" type="video/mp4"></video>')


# 3.Train Model

In [ ]:
model.learn(total_timesteps=500000)

# 4.Test mode

In [ ]:
record_video(model,env,video_filename="lunarlander.mp4",num_steps=1100,fps=60)
show_video(video_filename)

In [ ]:
model.save("lunarlander_model")

# 5.Use Model was saved

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
import gymnasium as gym
import numpy as np
import imageio
import os

env = gym.make("LunarLander-v3", render_mode="rgb_array")
model = PPO.load("/content/lunarlander_model1")

In [6]:
record_video(model,env,video_filename="lunarlander.mp4",num_steps=1100,fps=60)

✅ Video đã được lưu tại: lunarlander.mp4


In [ ]:
show_video(video_filename)

In [8]:
from stable_baselines3.common.evaluation import evaluate_policy
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=50)
print(f"Điểm trung bình: {mean_reward:.2f} +/- {std_reward:.2f}")

/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Điểm trung bình: 220.16 +/- 53.77
